In [1]:
import datetime
from dateutil.relativedelta import relativedelta
import requests
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read csv to get all the possible routes with Ryanair
ryanair_flights = pd.read_csv("/content/drive/MyDrive/RyanairFlights.csv", encoding = "ISO-8859-1")

In [4]:
ryanair_flights.head(5)

,Date,Route Origin Airport,Route Origin Name,Origin_Airport Name,Origin_City,Origin_Country,Origin_Airport Code,Origin_ICAO Code,Origin_Latitude,Origin_Longitude,...,Segment Flight Number,Segment Origin,Segment Destination,Segment Duration,Segment Distance (km),Segment Origin Time - Local,Segment Origin Time - UTC,Segment Destination Time - Local,Segment Destination Time - UTC,Request Time UTC
0,20180102,STN,London (Stansted),London Stansted Airport,London,United Kingdom,STN,EGSS,51.884998,0.235,...,FR 2205,STN,AAL,01:40,849.585165,20180102065000,20180102065000,20180102093000,20180102083000,2.017111e+13
1,20180104,STN,London (Stansted),London Stansted Airport,London,United Kingdom,STN,EGSS,51.884998,0.235,...,FR 2205,STN,AAL,01:40,849.585165,20180104065000,20180104065000,20180104093000,20180104083000,2.017111e+13
2,20180106,STN,London (Stansted),London Stansted Airport,London,United Kingdom,STN,EGSS,51.884998,0.235,...,FR 2205,STN,AAL,01:40,849.585165,20180106065000,20180106065000,20180106093000,20180106083000,2.017111e+13
3,20180107,STN,London (Stansted),London Stansted Airport,London,United Kingdom,STN,EGSS,51.884998,0.235,...,FR 2205,STN,AAL,01:40,849.585165,20180107071000,20180107071000,20180107095000,20180107085000,2.017111e+13
4,20180109,STN,London (Stansted),London Stansted Airport,London,United Kingdom,STN,EGSS,51.884998,0.235,...,FR 2205,STN,AAL,01:40,849.585165,20180109065000,20180109065000,20180109093000,20180109083000,2.017111e+13


In [9]:
# Extract airport abbreviation

ryanair_airports = ryanair_flights[['Segment Origin', 'Origin_Airport Name','Segment Destination']]
ryanair_airports = ryanair_airports.drop_duplicates()

In [17]:
studied_airports = ['STN', 'BCN', 'BLQ', 'EDI', 'LTN', 'SXF', 'VLC', 'AMS', 'BRU', 'IBZ', 'PSA', 'BVA', 'MAD', 'MXP']
restricted_flights = ryanair_airports[ryanair_airports['Segment Origin'].isin(studied_airports)]
restricted_flights = restricted_flights[restricted_flights['Segment Destination'].isin(studied_airports)]
restricted_flights.reset_index(inplace=True)
restricted_flights.drop(columns=restricted_flights.columns[0], axis=1, inplace=True)

In [19]:
routes = []
for i in range(restricted_flights.shape[0]):
    routes.append((restricted_flights["Segment Origin"][i], restricted_flights["Segment Destination"][i]))

In [21]:
print(len(routes))
restricted_flights.head(5)

80


,Segment Origin,Origin_Airport Name,Segment Destination
0,BLQ,Bologna Guglielmo Marconi Airport,BCN
1,BRU,Brussels Airport,BCN
2,BVA,Paris Beauvais TillÃ© Airport,BCN
3,EDI,Edinburgh Airport,BCN
4,IBZ,Ibiza Airport,BCN


In [22]:
restricted_flights.to_csv('/content/drive/MyDrive/data.csv')

In [ ]:

today = datetime.datetime.today()
master_df = pd.DataFrame(columns = ['Origin','Destination','Period','Date','Time','Price','Published Price'])

for keys, values in routes:
    two_week = today + relativedelta(days=+14)
    one_month = today + relativedelta(months=+1)
    four_month = today + relativedelta(months=+4)
    date_dict = {'Two Weeks':two_week,'One Month':one_month,'Four Months':four_month}
    for keys2, values2  in date_dict.items():
        date_in = values2.strftime('%Y-%m-%d')
        date_out = values2 + relativedelta(days=+7)
        date_out = date_out.strftime('%Y-%m-%d')
        url = 'https://www.ryanair.com/api/booking/v4/en-gb/availability?ADT=1&CHD=0&DateIn={}&DateOut={}&Destination={}&Disc=0&INF=0&Origin={}&TEEN=0&promoCode=&IncludeConnectingFlights=false&FlexDaysBeforeIn=2&FlexDaysIn=2&RoundTrip=true&FlexDaysBeforeOut=2&FlexDaysOut=2&ToUs=AGREED'.format(date_in, date_out, values, keys)
        content = str(requests.get(url).content)
        flight_keys, prices, published_prices = ([] for i in range(3)) 
        for i, _ in enumerate(content):
            needle1 = '"flightKey":"'
            needle2 = '"amount":'
            needle3 = '"publishedFare":'
            if content[i:i + len(needle1)] == needle1:
                flight_keys.append(content[i+13:i+200].split('~~","',1)[0])
            if content[i:i + len(needle2)] == needle2:
                prices.append(content[i+9:i+18].split(',',1)[0])
            if content[i:i + len(needle3)] == needle3:
                published_prices.append(content[i+16:i+25].split(',',1)[0])
        if len(flight_keys) == len(published_prices) == len(prices):
            for i in range(0, len(flight_keys)):
                time = flight_keys[i][-5:]
                date  = flight_keys[i][-16:-6]
                if flight_keys[i][-20:-17] == values:
                    origin = keys
                    destination = values
                else:
                    origin = values
                    destination = keys
                price = prices[i]
                published_price = published_prices[i]
                row = [origin,destination,keys2,date,time,price,published_price]
                master_df.loc[len(master_df)] = row
print(master_df)

In [25]:
master_df.to_csv('/content/drive/MyDrive/data.csv')